# Overall accuracy of Random Forest Classifier (rfc)
- Correct assignments:   738/1137 **(0.65)**
- Incorrect assignments: 399/1137 **(0.35)**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [6]:
pd.set_option('display.max_columns', None)

In [122]:
# open random forest classifier (rfc) results for the unknown dendritic classification
unknown_dendritic_rfc_results = pd.read_csv('unknown_dendritic_rfc_prediction_results.csv', index_col=[0])

# RFC setting used to generate these predictions
Parameters currently in use:

{'bootstrap': True,  
 'ccp_alpha': 0.0,  
 'class_weight': None,  
 'criterion': 'gini',  
 'max_depth': None,  
 'max_features': 'auto',  
 'max_leaf_nodes': None,  
 'max_samples': None,  
 'min_impurity_decrease': 0.0,  
 'min_samples_leaf': 1,  
 'min_samples_split': 2,  
 'min_weight_fraction_leaf': 0.0,  
 'n_estimators': 100,  
 'n_jobs': None,  
 'oob_score': False,  
 'random_state': 1,  
 'verbose': 0,  
 'warm_start': False}

In [21]:
# add new col cellid_str with string type to fix plotly error
unknown_dendritic_rfc_results['cellid_str'] = unknown_dendritic_rfc_results.loc[:, 'cellid']

# convert to str
unknown_dendritic_df = unknown_dendritic_rfc_results.astype({'cellid_str':'string'})
unknown_dendritic_df

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment,centroid_x,centroid_y,centroid_z,ctr_distance_to_soma,bbox_dir_x,bbox_dir_y,bbox_dir_z,bbox_distance,above_soma,within_threshold_1500,count,mean,std,min,25%,50%,75%,max,kurtosis,skew,rfc_predicted_compartment,cellid_str
0,1547760,211468,70706,45080,1364,70248,44874,1328,71030,45284,1393,648518346349523993,282824,180320,54560,2.716770,10.804325,Unknown dendritic,70864,42368,24,3029.110100,-782,-410,-65,885.352472,0,0,196.0,6307.454632,5294.853299,0.0,2791.498779,4378.841631,8158.576625,26607.713618,5.541097,2.220513,Basal,648518346349523993
1,1774335,516120,74524,41666,984,73392,41388,903,75434,41838,1093,648518346349523993,298096,166664,39360,6.773440,28.109598,Unknown dendritic,70864,42368,24,3848.376801,-2042,-450,-190,2099.610440,1,0,196.0,7566.268937,5808.272535,0.0,3782.081357,5032.257927,9528.231655,27537.681311,4.937059,2.058686,Basal,648518346349523993
2,1662490,3004,71398,43960,1139,71356,43932,1135,71440,43992,1141,648518346349523993,285592,175840,45560,0.102391,2.866273,Unknown dendritic,70864,42368,24,2015.650019,-84,-60,-6,103.402128,0,0,196.0,6073.269636,5674.473901,0.0,2294.075603,3740.158135,8079.002508,26774.752081,5.567801,2.236830,Basal,648518346349523993
3,1547394,879336,70958,45388,1891,70558,44828,1662,71340,46262,2157,648518346349523993,283832,181552,75640,11.790898,51.080975,Unknown dendritic,70864,42368,24,3551.749569,-782,-1434,-495,1706.723469,0,0,196.0,6512.100850,5110.648731,0.0,3205.207022,4563.925525,8208.348254,26230.706376,5.448838,2.191350,Basal,648518346349523993
4,1541327,547340,70932,42378,866,70804,41794,785,71188,42902,946,648518346349523993,283728,169512,34640,6.168662,18.879337,Unknown dendritic,70864,42368,24,844.800568,-384,-1108,-161,1183.655778,0,1,196.0,6163.671139,6160.340209,0.0,1504.873059,3839.307731,8101.049803,28171.214315,5.607736,2.249625,Basal,648518346349523993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132,545422,261204,51806,45212,1622,51278,45042,1610,52394,45356,1635,648518346349492197,207224,180848,64880,3.089287,10.412219,Unknown dendritic,67920,39584,2148,17076.652365,-1116,-314,-25,1159.602087,0,0,190.0,20341.084297,9787.632079,0.0,15003.853240,17033.270424,25006.554119,51026.602758,2.268589,1.241689,Basal,648518346349492197
1133,633845,366232,54084,38652,1164,53490,38578,1120,54614,38726,1212,648518346349492197,216336,154608,46560,4.573971,17.190795,Unknown dendritic,67920,39584,2148,13902.221981,-1124,-148,-92,1137.428679,1,0,190.0,18092.233043,10010.550310,0.0,12552.559723,14057.313489,22983.893509,48606.099628,2.794063,1.424764,Basal,648518346349492197
1134,973784,2897496,62588,42158,510,59768,41724,74,65388,42622,1025,648518346349492197,250352,168632,20400,30.240936,79.372160,Unknown dendritic,67920,39584,2148,6143.186795,-5620,-898,-951,5770.199737,0,0,190.0,10991.697855,8796.647867,0.0,5289.077021,6715.396244,14679.910696,40098.947779,4.698357,2.012254,Basal,648518346349492197
1135,1310832,752504,66324,41486,1332,66142,41328,1270,66814,41942,1412,648518346349492197,265296,165944,53280,7.230910,16.087532,Unknown dendritic,67920,39584,2148,2613.556198,-672,-614,-142,921.273032,0,0,190.0,8718.497145,8399.523429,0.0,2469.147283,5702.917607,12741.309694,36325.865895,4.853222,2.070600,Basal,648518346349492197


In [22]:
unknown_dendritic_df.cellid_str.dtype

string[python]

In [25]:
list_of_cellids = unknown_dendritic_df.cellid.unique()
list_of_cellids

array([648518346349523993, 648518346349533350, 648518346349538475,
       648518346349539845, 648518346349539864, 648518346349536748,
       648518346349532067, 648518346349534360, 648518346349534614,
       648518346349535102, 648518346349539097, 648518346349537047,
       648518346349531254, 648518346349539890, 648518346349538133,
       648518346349538157, 648518346349540048, 648518346349537071,
       648518346349537513, 648518346349536809, 648518346349536679,
       648518346349535192, 648518346349538235, 648518346349524971,
       648518346349493472, 648518346349537883, 648518346349492197],
      dtype=int64)

In [26]:
len(unknown_dendritic_df.cellid.unique())

27

In [27]:
list_of_cellids[0]

648518346349523993

In [49]:
unknown_dendritic_df.rfc_predicted_compartment.value_counts()

Basal     1089
Apical      48
Name: rfc_predicted_compartment, dtype: int64

# Look at mitos assigned to apical compartment

### View all mito assigned to apical dendrite compartment by the random forest classifier in [Neuroglancer](https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.05%2C%22segments%22:%5B%22648518346349533350%22%2C%22648518346349534614%22%2C%22648518346349535102%22%2C%22648518346349537071%22%2C%22648518346349538157%22%2C%22648518346349538235%22%2C%22648518346349539845%22%2C%22648518346349539864%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%221299116%22%2C%221421155%22%2C%221421377%22%2C%221421392%22%2C%221421455%22%2C%221421579%22%2C%221534618%22%2C%221883096%22%2C%221884146%22%2C%221884228%22%2C%221890449%22%2C%221890626%22%2C%222010133%22%2C%222010351%22%2C%222010474%22%2C%222010681%22%2C%222010754%22%2C%222592629%22%2C%222592661%22%2C%222593432%22%2C%222708427%22%2C%222709775%22%2C%222817428%22%2C%222822118%22%2C%222824088%22%2C%222824625%22%2C%222825064%22%2C%222825184%22%2C%222826530%22%2C%222828087%22%2C%222828453%22%2C%222829461%22%2C%222933643%22%2C%222935817%22%2C%223044661%22%2C%223153502%22%2C%223153917%22%2C%223158686%22%2C%223273270%22%2C%223815340%22%2C%223815405%22%2C%223927566%22%2C%223927604%22%2C%224037974%22%2C%224042999%22%2C%224043085%22%2C%22639983%22%2C%22746709%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B83222.921875%2C52981.34765625%2C824.9962768554688%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B-0.00825042650103569%2C0.06130112707614899%2C-0.0012821174459531903%2C0.9980843663215637%5D%2C%22perspectiveZoom%22:1879.3613276615747%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D). 

In [131]:
# generate df of apical assignments only
unknown_dendritic_df_apical = unknown_dendritic_df[unknown_dendritic_df.rfc_predicted_compartment == "Apical"]
unknown_dendritic_df_apical

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment,centroid_x,centroid_y,centroid_z,ctr_distance_to_soma,bbox_dir_x,bbox_dir_y,bbox_dir_z,bbox_distance,above_soma,within_threshold_1500,count,mean,std,min,25%,50%,75%,max,kurtosis,skew,rfc_predicted_compartment,cellid_str
40,3153917,73264,98910,42676,292,98728,42456,286,99152,42854,299,648518346349533350,395640,170704,11680,1.189456,7.554254,Unknown dendritic,104320,46432,596,6593.030563,-424,-398,-13,581.677746,1,0,616.0,11416.906494,7411.822952,0.0,6727.639920,8201.872356,14641.036020,48735.148507,5.765466,2.277201,Apical,648518346349533350
45,3153502,18488,98430,42284,277,98362,42230,275,98500,42340,280,648518346349533350,393720,169136,11080,0.305789,2.015650,Unknown dendritic,104320,46432,596,7211.086257,-138,-110,-5,176.547444,1,0,616.0,11832.314159,7293.717998,0.0,7219.083880,8736.819709,15024.900608,48377.432435,5.592162,2.230453,Apical,648518346349533350
49,3158686,380512,99928,43504,295,99144,43104,278,100522,43850,323,648518346349533350,399712,174016,11800,5.004979,20.864031,Unknown dendritic,104320,46432,596,5287.102136,-1378,-746,-45,1567.617619,1,0,616.0,10575.538478,7697.335403,0.0,5536.285417,7096.480159,13792.108405,49512.016289,6.087650,2.363426,Apical,648518346349533350
51,2935817,495544,95990,41568,302,94930,41288,287,97150,41836,320,648518346349533350,383960,166272,12080,6.642364,28.756212,Unknown dendritic,104320,46432,596,9650.587132,-2220,-548,-33,2286.874067,1,0,616.0,13383.808436,6791.101722,0.0,9326.406974,11023.956638,16603.749720,46243.946458,4.666881,1.974108,Apical,648518346349533350
242,3044661,207464,97256,39404,1728,97046,39194,1723,97498,39780,1739,648518346349539845,389024,157616,69120,2.320565,6.995585,Unknown dendritic,103776,47792,1558,10625.339712,-452,-586,-16,740.240501,1,0,642.0,14212.385693,6676.071462,0.0,10204.131048,12186.968572,17460.463232,38298.473077,2.888592,1.451967,Apical,648518346349539845
267,2824088,154308,92448,42592,1057,92336,42264,1041,92558,42962,1072,648518346349539864,369792,170368,42280,2.161302,10.216393,Unknown dendritic,89520,48128,1306,6267.573773,-222,-698,-31,733.109132,1,0,692.0,12124.753296,8418.264793,0.0,5761.640956,8966.776198,17051.052967,46516.807543,4.871234,2.068003,Apical,648518346349539864
270,2817428,164012,92704,38454,1310,92454,38100,1281,92858,38888,1332,648518346349539864,370816,153816,52400,2.269310,10.467878,Unknown dendritic,89520,48128,1306,10184.505290,-404,-788,-51,886.995490,1,0,692.0,15218.690521,8320.990309,0.0,9234.084927,12096.144710,19780.989855,48497.803672,4.018805,1.816446,Apical,648518346349539864
271,2822118,84004,92764,40722,1179,92658,40516,1168,92862,40952,1192,648518346349539864,371056,162888,47160,1.210039,6.049592,Unknown dendritic,89520,48128,1306,8086.315663,-204,-436,-24,481.962654,1,0,692.0,13493.368989,8394.034902,0.0,7413.485557,10239.521530,17927.681700,47558.885185,4.618162,1.986642,Apical,648518346349539864
273,2829461,33316,93310,43564,1036,93160,43534,1030,93462,43598,1044,648518346349539864,373240,174256,41440,0.578427,4.201140,Unknown dendritic,89520,48128,1306,5938.610612,-302,-64,-14,309.024271,1,0,692.0,11829.422489,8388.942946,0.0,5525.426249,8428.337231,16514.597367,46950.277539,5.095113,2.125203,Apical,648518346349539864
274,2828087,36068,91994,44236,1015,91896,44120,1006,92074,44342,1024,648518346349539864,367976,176944,40600,0.631363,4.661465,Unknown dendritic,89520,48128,1306,4620.932914,-178,-222,-18,285.117520,1,0,692.0,11019.793270,8417.165672,0.0,4479.936381,7839.023789,16231.011178,45480.815120,5.052202,2.124867,Apical,648518346349539864


In [123]:
# how many mitos were assigned by the rfc to the apical compartment
len(unknown_dendritic_df_apical)

48

In [124]:
# these mitos come from which cellids
unknown_dendritic_df_apical.cellid.unique()

array([648518346349533350, 648518346349539845, 648518346349539864,
       648518346349534614, 648518346349535102, 648518346349538157,
       648518346349537071, 648518346349538235], dtype=int64)

In [125]:
# how many cellids have mitos assigned to the apical compartment
len(unknown_dendritic_df_apical.cellid.unique())

8

In [126]:
# generate a list of the mito ids that were assigned by the rfc to the apical compartment
unknown_dendritic_df_apical.mito_id.unique()

array([3153917, 3153502, 3158686, 2935817, 3044661, 2824088, 2817428,
       2822118, 2829461, 2828087, 2825184, 2824625, 2825064, 2826530,
       2828453, 4037974, 3815340, 3927566, 3815405, 3927604, 4043085,
       4042999, 2933643, 2592661, 2708427, 3273270, 2709775, 2592629,
       2593432, 1421455, 1421392, 1421155, 1421377, 1534618, 1421579,
        639983, 1890626, 2010133, 1890449, 1884146, 2010474, 2010681,
       2010754,  746709, 1884228, 1883096, 1299116, 2010351], dtype=int64)

In [127]:
# math check
len(unknown_dendritic_df_apical.mito_id.unique())

48

## Reveal apical assignments in Neuroglancer by cellid
This is used to manually verify if the rfc was correct in its assignment  
There are 8 cellids associated with these apical assignments  
Manually change cellid_iloc from 0 to 7 to view each neuron one by one

In [113]:
cellid_iloc = 7

list_of_cellids = unknown_dendritic_df_apical.cellid.unique()
mito = unknown_dendritic_df_apical
lookup = list_of_cellids[cellid_iloc]
query = mito[mito["cellid"] == lookup]
query_list = query["mito_id"].to_list()
spacer = "%22%2C%22"
query_list_join = spacer.join(map(str,query_list))
baseURL = 'https://neuromancer-seung-import.appspot.com/'
nglink_a = '#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22'
nglink_b = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%22'
nglink_c = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B83222.921875%2C52981.34765625%2C824.9962768554688%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B-0.00825042650103569%2C0.06130112707614899%2C-0.0012821174459531903%2C0.9980843663215637%5D%2C%22perspectiveZoom%22:3618.7659948513424%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D'
neuroglancerURL = baseURL + nglink_a + str(lookup) + nglink_b + query_list_join + nglink_c
print(neuroglancerURL)

https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349538235%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmenta

In [128]:
# how many mitos that were classified as belonging to the apical compartment are in this cell
len(query_list)

12

## Check in Neuroglancer for difficult to verify cases (apical version)
Use this to visualize mitos compartment assignments as performed by the Turner et al Cell 2022 dataset

In [148]:
# Generate ng view for cellid ids of interest
# This was used for further inspection of the following cellids, which were more difficult to manually verify
# 648518346349539864; I classified this as a basal dendrite; it is very close to the apical dendrite which made it difficult to classify properly
# 648518346349538157; this should be classified as basal not apical; also of note is that this dendrite has most of its mito segmented as soma mitos (but clearly are not); this would be an errorenous assignment in the original dataset by Turner et al.

cellid_of_interest = 648518346349538157
mitoabovesoma = pd.read_csv('../data/mito_above_soma.csv', index_col=[0])
mitoabovesoma_basal = mitoabovesoma[mitoabovesoma['compartment'] == 'Basal']
mitoabovesoma_apical = mitoabovesoma[mitoabovesoma['compartment'] == 'Apical']
mitoabovesoma_somatic = mitoabovesoma[mitoabovesoma['compartment'] == 'Somatic']
mitoabovesoma_unknowndendritic = mitoabovesoma[mitoabovesoma['compartment'] == 'Unknown dendritic']
mitoabovesoma_unknown = mitoabovesoma[mitoabovesoma['compartment'] == 'Unknown']
mitoabovesoma_axonal = mitoabovesoma[mitoabovesoma['compartment'] == 'Axonal']

In [149]:
# enter the compartment of interest in the line below; choose 'mitoabovesoma' for all compartments
mito = mitoabovesoma_somatic

lookup = cellid_of_interest
query = mito[mito["cellid"] == lookup]
query_list = query["mito_id"].to_list()
spacer = "%22%2C%22"
query_list_join = spacer.join(map(str,query_list))
baseURL = 'https://neuromancer-seung-import.appspot.com/'
nglink_a = '#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22'
nglink_b = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%22'
nglink_c = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B83222.921875%2C52981.34765625%2C824.9962768554688%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B-0.00825042650103569%2C0.06130112707614899%2C-0.0012821174459531903%2C0.9980843663215637%5D%2C%22perspectiveZoom%22:3618.7659948513424%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D'
neuroglancerURL = baseURL + nglink_a + str(lookup) + nglink_b + query_list_join + nglink_c
print(neuroglancerURL)

https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349538157%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmenta

## Results of manual validation of apical assignments

11/48 (0.229) classified as apical were true apical upon manual validation  
37/48 (0.771) classified as apical were actually basal dendrites as determined by manual inspection

In [217]:
# the manual inspection was tabulated in Excel, exported to csv in the file below
manual_validation_apical = pd.read_csv('round1_rfc_assignment_manual_validation_apical.csv')
manual_validation_apical

,cellid_str,mito_id,rfc_assignment,manual_inspection,correct_assignment,notes,ng_link
0,648518346349533350,3153917,Apical,Basal,False,apical dendrite is rotated off the vertical axis,https://neuromancer-seung-import.appspot.com/#...
1,648518346349533350,3153502,Apical,Basal,False,apical dendrite is rotated off the vertical axis,https://neuromancer-seung-import.appspot.com/#...
2,648518346349533350,2935817,Apical,Basal,False,apical dendrite is rotated off the vertical axis,https://neuromancer-seung-import.appspot.com/#...
3,648518346349533350,3158686,Apical,Basal,False,apical dendrite is rotated off the vertical axis,https://neuromancer-seung-import.appspot.com/#...
4,648518346349539845,3044661,Apical,Apical,True,correct assignment,https://neuromancer-seung-import.appspot.com/#...
5,648518346349539845,2825184,Apical,Basal,False,this dendrite is close to the apical dendrite,https://neuromancer-seung-import.appspot.com/#...
6,648518346349539845,2825064,Apical,Basal,False,this dendrite is close to the apical dendrite,https://neuromancer-seung-import.appspot.com/#...
7,648518346349539845,2826530,Apical,Basal,False,this dendrite is close to the apical dendrite,https://neuromancer-seung-import.appspot.com/#...
8,648518346349539845,2828453,Apical,Basal,False,this dendrite is close to the apical dendrite,https://neuromancer-seung-import.appspot.com/#...
9,648518346349539845,2824088,Apical,Basal,False,this dendrite is close to the apical dendrite,https://neuromancer-seung-import.appspot.com/#...


In [218]:
# enter iloc value of interest if you wish to pull up the neuroglancer view for a row of interest
iloc_value = 5
print(manual_validation_apical.ng_link[iloc_value])

https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22colorSeed%22:4051816633%2C%22segments%22:%5B%22648518346349539864%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22colorSeed%22:4273571984%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-ar

# Look at mitos assigned to basal compartment
This is a much large list of mitos (n = 1089)

In [70]:
# generate df of basal assignments only
unknown_dendritic_df_basal = unknown_dendritic_df[unknown_dendritic_df.rfc_predicted_compartment == "Basal"]
unknown_dendritic_df_basal

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment,centroid_x,centroid_y,centroid_z,ctr_distance_to_soma,bbox_dir_x,bbox_dir_y,bbox_dir_z,bbox_distance,above_soma,within_threshold_1500,count,mean,std,min,25%,50%,75%,max,kurtosis,skew,rfc_predicted_compartment,cellid_str
0,1547760,211468,70706,45080,1364,70248,44874,1328,71030,45284,1393,648518346349523993,282824,180320,54560,2.716770,10.804325,Unknown dendritic,70864,42368,24,3029.110100,-782,-410,-65,885.352472,0,0,196.0,6307.454632,5294.853299,0.0,2791.498779,4378.841631,8158.576625,26607.713618,5.541097,2.220513,Basal,648518346349523993
1,1774335,516120,74524,41666,984,73392,41388,903,75434,41838,1093,648518346349523993,298096,166664,39360,6.773440,28.109598,Unknown dendritic,70864,42368,24,3848.376801,-2042,-450,-190,2099.610440,1,0,196.0,7566.268937,5808.272535,0.0,3782.081357,5032.257927,9528.231655,27537.681311,4.937059,2.058686,Basal,648518346349523993
2,1662490,3004,71398,43960,1139,71356,43932,1135,71440,43992,1141,648518346349523993,285592,175840,45560,0.102391,2.866273,Unknown dendritic,70864,42368,24,2015.650019,-84,-60,-6,103.402128,0,0,196.0,6073.269636,5674.473901,0.0,2294.075603,3740.158135,8079.002508,26774.752081,5.567801,2.236830,Basal,648518346349523993
3,1547394,879336,70958,45388,1891,70558,44828,1662,71340,46262,2157,648518346349523993,283832,181552,75640,11.790898,51.080975,Unknown dendritic,70864,42368,24,3551.749569,-782,-1434,-495,1706.723469,0,0,196.0,6512.100850,5110.648731,0.0,3205.207022,4563.925525,8208.348254,26230.706376,5.448838,2.191350,Basal,648518346349523993
4,1541327,547340,70932,42378,866,70804,41794,785,71188,42902,946,648518346349523993,283728,169512,34640,6.168662,18.879337,Unknown dendritic,70864,42368,24,844.800568,-384,-1108,-161,1183.655778,0,1,196.0,6163.671139,6160.340209,0.0,1504.873059,3839.307731,8101.049803,28171.214315,5.607736,2.249625,Basal,648518346349523993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132,545422,261204,51806,45212,1622,51278,45042,1610,52394,45356,1635,648518346349492197,207224,180848,64880,3.089287,10.412219,Unknown dendritic,67920,39584,2148,17076.652365,-1116,-314,-25,1159.602087,0,0,190.0,20341.084297,9787.632079,0.0,15003.853240,17033.270424,25006.554119,51026.602758,2.268589,1.241689,Basal,648518346349492197
1133,633845,366232,54084,38652,1164,53490,38578,1120,54614,38726,1212,648518346349492197,216336,154608,46560,4.573971,17.190795,Unknown dendritic,67920,39584,2148,13902.221981,-1124,-148,-92,1137.428679,1,0,190.0,18092.233043,10010.550310,0.0,12552.559723,14057.313489,22983.893509,48606.099628,2.794063,1.424764,Basal,648518346349492197
1134,973784,2897496,62588,42158,510,59768,41724,74,65388,42622,1025,648518346349492197,250352,168632,20400,30.240936,79.372160,Unknown dendritic,67920,39584,2148,6143.186795,-5620,-898,-951,5770.199737,0,0,190.0,10991.697855,8796.647867,0.0,5289.077021,6715.396244,14679.910696,40098.947779,4.698357,2.012254,Basal,648518346349492197
1135,1310832,752504,66324,41486,1332,66142,41328,1270,66814,41942,1412,648518346349492197,265296,165944,53280,7.230910,16.087532,Unknown dendritic,67920,39584,2148,2613.556198,-672,-614,-142,921.273032,0,0,190.0,8718.497145,8399.523429,0.0,2469.147283,5702.917607,12741.309694,36325.865895,4.853222,2.070600,Basal,648518346349492197


In [133]:
# how many mitos were assigned by the rfc to the apical compartment
len(unknown_dendritic_df_basal)

1089

In [135]:
# these mitos come from which cellids
unknown_dendritic_df_basal.cellid.unique()

array([648518346349523993, 648518346349533350, 648518346349538475,
       648518346349539845, 648518346349539864, 648518346349536748,
       648518346349532067, 648518346349534360, 648518346349534614,
       648518346349535102, 648518346349539097, 648518346349537047,
       648518346349531254, 648518346349539890, 648518346349538133,
       648518346349540048, 648518346349537071, 648518346349537513,
       648518346349536809, 648518346349536679, 648518346349535192,
       648518346349538235, 648518346349524971, 648518346349493472,
       648518346349537883, 648518346349492197], dtype=int64)

In [136]:
# how many cellids have mitos assigned to the apical compartment
len(unknown_dendritic_df_basal.cellid.unique())

26

In [138]:
# generate a list of the mito ids that were assigned by the rfc to the apical compartment
unknown_dendritic_df_basal.mito_id.unique()

array([1547760, 1774335, 1662490, ...,  973784, 1310832, 1305695],
      dtype=int64)

In [140]:
# math check
len(unknown_dendritic_df_basal.mito_id.unique())

1089

## Reveal basal assignments in Neuroglancer by cellid
This is used to manually verify if the rfc was correct in its assignment  
There are 26 cellids associated with these basal assignments  
Manually change cellid_iloc from 0 to 25 to view each neuron one by one

In [488]:
cellid_iloc = 17

list_of_cellids = unknown_dendritic_df_basal.cellid.unique()
mito = unknown_dendritic_df_basal
lookup = list_of_cellids[cellid_iloc]
query = mito[mito["cellid"] == lookup]
query_list = query["mito_id"].to_list()
spacer = "%22%2C%22"
query_list_join = spacer.join(map(str,query_list))
baseURL = 'https://neuromancer-seung-import.appspot.com/'
nglink_a = '#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22'
nglink_b = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%22'
nglink_c = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B83222.921875%2C52981.34765625%2C824.9962768554688%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B-0.00825042650103569%2C0.06130112707614899%2C-0.0012821174459531903%2C0.9980843663215637%5D%2C%22perspectiveZoom%22:3618.7659948513424%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D'
neuroglancerURL = baseURL + nglink_a + str(lookup) + nglink_b + query_list_join + nglink_c
print(neuroglancerURL)

https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349537513%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmenta

In [489]:
# how many mitos that were classified as belonging to the apical compartment are in this cell
len(query_list)

13

In [490]:
# uncomment if you need a list of mito ids

np.array(query_list)

array([1852507, 1852441, 1725469, 1852659, 1852316, 1852546, 1724749,
       1725445, 1846859, 1852488, 1724948, 1846747, 1725308])

In [471]:
# uncomment to view dataframe of mitos

# query

## Check in Neuroglancer for difficult to verify cases (basal version)
Use this to visualize mitos compartment assignments as performed by the Turner et al Cell 2022 dataset

In [472]:
# Generate ng view for cellid ids of interest
# This was used for further inspection of the following cellids, which were more difficult to manually verify
# cellid 648518346349523993; this is an edge of volume neuron; the dendrites all appear to be basal based on non-vertical orientation but it is impossible to know for certain

cellid_of_interest = list_of_cellids[cellid_iloc] # or enter a specific cellid of interest
mitoabovesoma = pd.read_csv('../data/mito_above_soma.csv', index_col=[0])
mitoabovesoma_basal = mitoabovesoma[mitoabovesoma['compartment'] == 'Basal']
mitoabovesoma_apical = mitoabovesoma[mitoabovesoma['compartment'] == 'Apical']
mitoabovesoma_somatic = mitoabovesoma[mitoabovesoma['compartment'] == 'Somatic']
mitoabovesoma_unknowndendritic = mitoabovesoma[mitoabovesoma['compartment'] == 'Unknown dendritic']
mitoabovesoma_unknown = mitoabovesoma[mitoabovesoma['compartment'] == 'Unknown']
mitoabovesoma_axonal = mitoabovesoma[mitoabovesoma['compartment'] == 'Axonal']

In [473]:
# enter the compartment of interest in the line below; choose 'mitoabovesoma' for all compartments
# if you get a four-panel view in neuroglancer, that means that there is no mito assignments for that particular compartment
mito = mitoabovesoma_apical

lookup2 = cellid_of_interest
query2 = mito[mito["cellid"] == lookup2]
query_list2 = query2["mito_id"].to_list()
spacer = "%22%2C%22"
query_list_join2 = spacer.join(map(str,query_list2))
baseURL = 'https://neuromancer-seung-import.appspot.com/'
nglink_a = '#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22'
nglink_b = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%22'
nglink_c = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B83222.921875%2C52981.34765625%2C824.9962768554688%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B-0.00825042650103569%2C0.06130112707614899%2C-0.0012821174459531903%2C0.9980843663215637%5D%2C%22perspectiveZoom%22:3618.7659948513424%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D'
neuroglancerURL = baseURL + nglink_a + str(lookup2) + nglink_b + query_list_join2 + nglink_c
print(neuroglancerURL)

https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349538235%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmenta

In [474]:
# uncomment if you need to look up data on the mito list

# query2


## Results of manual validation of basal assignments

In [475]:
print('index '+str(cellid_iloc)+': cellid '+str(lookup)+'; number of mitos: '+str(len(query_list)))

index 21: cellid 648518346349538235; number of mitos: 86


### All mito that are correctly classified as basal by the rfc prediction  
- index 0: cellid 648518346349523993; number of mitos: 40  
- index 1: cellid 648518346349533350; number of mitos: 11  
- index 2: cellid 648518346349538475; number of mitos: 33  
- index 4: cellid 648518346349539864; number of mitos: 22
- index 8: cellid 648518346349534614; number of mitos: 72; edge of volume neuron, although these appear to all be correctly identified as basal, based on the orientation of the processes
- index 9: cellid 648518346349535102; number of mitos: 52; ditto
- index 10: cellid 648518346349539097; number of mitos: 80; ditto, appears to be basal based on orientation
- index 11: cellid 648518346349537047; number of mitos: 3; could be an axon hillock but looks more likely to be a basal dendrite
- index 14: cellid 648518346349538133; number of mitos: 25; edge of volume neuron but seems clearly basal by orientation
- index 15: cellid 648518346349540048; number of mitos: 30; edge of volume neuron but seems clearly basal by orientation
- index 20: cellid 648518346349535192; number of mitos: 47
- index 22: cellid 648518346349524971; number of mitos: 7
- index 23: cellid 648518346349493472; number of mitos: 29; very little of soma is present however, this seems clearly basal by orientation
- index 24: cellid 648518346349537883; number of mitos: 4; this appears to be basal even though the process is truncated and very close to the apical dendrite 
- index 25: cellid 648518346349492197; number of mitos: 66; edge neuron, this appears to be basal based on downward orientation

### Total classifed as basal and are confirmed as basal: 521
40+11+33+22+72+52+80+3+25+30+47+7+29+4+66 =

### Incorrect assignment to basal that should have been classified as apical
- index 19: cellid 648518346349536679; number of mitos: 33; this appears to be an apical dendrite even though this neuron is near the volume edge; see in [Neuroglancer](https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349536679%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%221085129%22%2C%221086152%22%2C%221087784%22%2C%221186954%22%2C%221187020%22%2C%221187079%22%2C%221191964%22%2C%221197285%22%2C%221197452%22%2C%221197826%22%2C%221198898%22%2C%221307982%22%2C%221308002%22%2C%221308138%22%2C%221309292%22%2C%221312472%22%2C%221420168%22%2C%221421416%22%2C%221530291%22%2C%221651144%22%2C%221770295%22%2C%221884059%22%2C%22634451%22%2C%22750834%22%2C%22751611%22%2C%22752087%22%2C%22752228%22%2C%22853857%22%2C%22854537%22%2C%22966384%22%2C%22973324%22%2C%22974207%22%2C%22974881%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B65565.5703125%2C43397.0703125%2C327.09771728515625%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B0.025046376511454582%2C0.9582146406173706%2C-0.046196699142456055%2C0.2811819016933441%5D%2C%22perspectiveZoom%22:904.6914987128356%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D)

### Total classified as basal but are actually apical: 33 

### Difficult cases; these will likely need to remain classified as unknown dendritic
- index 5: cellid 648518346349536748; number of mitos: 16; Morphologically, this process looks like it is an axon (it is branched from the putative axon hillock); the synapse visualizer shows that there are no pre-syn suggesting it is a dendrite but morphologically, it looks like an axon; this should not have been classified as 'unknown dendritic' by the Turner algorithm; see in [Neuroglancer](https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349536748%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%221110444%22%2C%221110781%22%2C%221226503%22%2C%221227495%22%2C%22551428%22%2C%22657839%22%2C%22657973%22%2C%22659171%22%2C%22659275%22%2C%22659488%22%2C%22771854%22%2C%22771933%22%2C%22772690%22%2C%22880563%22%2C%22880606%22%2C%22998238%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B56946.66015625%2C50561.671875%2C1721.8668212890625%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B0.062064070254564285%2C0.0353650264441967%2C-0.02572326734662056%2C0.9971136450767517%5D%2C%22perspectiveZoom%22:452.3457493564178%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22cell_segmentation_v185%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D)  
- index 6: cellid 648518346349532067; number of mitos: 9; An edge of the volume neuron; by orientation looks like an axon hillock but there is not enough of the process to determine if it is an axon or a basal dendrite; see in [Neuroglancer](https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349532067%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%224118518%22%2C%224240389%22%2C%224240391%22%2C%224240423%22%2C%224240472%22%2C%224240479%22%2C%224240494%22%2C%224240544%22%2C%224240545%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B116143.4453125%2C73643.15625%2C2108.47314453125%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22showAxisLines%22:false%2C%22perspectiveOrientation%22:%5B0.18531009554862976%2C-0.06271055340766907%2C-0.06994727998971939%2C0.978179395198822%5D%2C%22perspectiveZoom%22:854.3606298156484%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22cell_segmentation_v185%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D)  
- index 7: cellid 648518346349534360; number of mitos: 2; edge of volume; looks like an apical dendrite but hard to determine for certain; see in [Neuroglancer](https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349534360%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%222122260%22%2C%222122556%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B83477.8359375%2C42423.28515625%2C1908.1787109375%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B0.0809904932975769%2C0.6224336624145508%2C-0.07208359241485596%2C0.7751263380050659%5D%2C%22perspectiveZoom%22:904.6914987128356%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D)
- index 13: cellid 648518346349539890; number of mitos: 93; near the edge of the volume which complicates interpretation; looks like these should be basal dendrites by orientation, but the morphology of the cell suggests the process in question is the apical dendrite turned 90 degrees on its side; see in [Neuroglancer](https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349539890%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%221006282%22%2C%222021236%22%2C%222021333%22%2C%222021783%22%2C%222135928%22%2C%222135953%22%2C%222135954%22%2C%222136080%22%2C%222136156%22%2C%222136362%22%2C%222136420%22%2C%222136495%22%2C%222136590%22%2C%222136782%22%2C%222136995%22%2C%222137180%22%2C%222239838%22%2C%222251557%22%2C%222253984%22%2C%222254049%22%2C%222254379%22%2C%222254401%22%2C%222259029%22%2C%222259111%22%2C%222264219%22%2C%222352170%22%2C%222355691%22%2C%222361034%22%2C%222363756%22%2C%222364250%22%2C%222367216%22%2C%222368091%22%2C%222368496%22%2C%222368520%22%2C%222368979%22%2C%222369148%22%2C%222380590%22%2C%222385236%22%2C%222385603%22%2C%222386207%22%2C%222386378%22%2C%222390280%22%2C%222390334%22%2C%222396073%22%2C%222396587%22%2C%222401827%22%2C%222477607%22%2C%222481757%22%2C%222482709%22%2C%222483302%22%2C%222484670%22%2C%222485248%22%2C%222485468%22%2C%222486167%22%2C%222486265%22%2C%222489785%22%2C%222495101%22%2C%222496793%22%2C%222497170%22%2C%222500182%22%2C%222500410%22%2C%222502793%22%2C%222503262%22%2C%222508285%22%2C%222508645%22%2C%222522570%22%2C%222527848%22%2C%222585440%22%2C%222599600%22%2C%222599929%22%2C%222608454%22%2C%222621566%22%2C%222622100%22%2C%222622898%22%2C%222646013%22%2C%222722967%22%2C%222724242%22%2C%222726304%22%2C%222730059%22%2C%222730077%22%2C%222730236%22%2C%222730708%22%2C%222844858%22%2C%222955208%22%2C%223070115%22%2C%223070528%22%2C%223070859%22%2C%223071664%22%2C%223289499%22%2C%223289985%22%2C%223290929%22%2C%223397519%22%2C%22888340%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B82019.09375%2C45297.18359375%2C324.25726318359375%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B0.11960885673761368%2C0.9729101657867432%2C0.10051172226667404%2C0.17040212452411652%5D%2C%22perspectiveZoom%22:904.6914987128356%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D)
- index 16: cellid 648518346349537071; number of mitos: 11; appears to be basal but not enough of soma to know
- index 17: cellid 648518346349537513; number of mitos: 13; this looks like an axon process coming off of a basal dendrite instead of the soma; these mitos were classified as unknown dendritic, which is true if this is a dendrite, but it looks like an axon hillock coming off of a axon-carrying basal dendrite; see [Neuroglancer](https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349537513%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%221724749%22%2C%221724948%22%2C%221725308%22%2C%221725445%22%2C%221725469%22%2C%221846747%22%2C%221846859%22%2C%221852316%22%2C%221852441%22%2C%221852488%22%2C%221852507%22%2C%221852546%22%2C%221852659%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B72041.78125%2C70495.5859375%2C1970.5035400390625%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22showAxisLines%22:false%2C%22perspectiveOrientation%22:%5B0.05819685757160187%2C-0.9912160634994507%2C0.0335722379386425%2C0.11391554772853851%5D%2C%22perspectiveZoom%22:452.3457493564178%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D)
- index 18: cellid 648518346349536809; number of mitos: 55; edge of volume neuron; the process appears to be basal but hard to know for certain; see in [Neuroglancer](https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349536809%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%221073030%22%2C%221075751%22%2C%221078272%22%2C%221078803%22%2C%221079067%22%2C%221081467%22%2C%221085232%22%2C%221088514%22%2C%221089004%22%2C%221089497%22%2C%221093722%22%2C%221193911%22%2C%221194291%22%2C%221194472%22%2C%221194477%22%2C%221197237%22%2C%221197377%22%2C%221197816%22%2C%221201647%22%2C%221205828%22%2C%221306066%22%2C%221307583%22%2C%221307693%22%2C%221312362%22%2C%221312708%22%2C%221312716%22%2C%221313003%22%2C%221313531%22%2C%221425896%22%2C%221433259%22%2C%221434291%22%2C%221544548%22%2C%221544797%22%2C%221548236%22%2C%221548253%22%2C%221548291%22%2C%221548311%22%2C%221548544%22%2C%221548810%22%2C%221548818%22%2C%22542141%22%2C%22553648%22%2C%22554169%22%2C%22655383%22%2C%22660209%22%2C%22739777%22%2C%22746402%22%2C%22746584%22%2C%22854127%22%2C%22854899%22%2C%22873631%22%2C%22974454%22%2C%22978185%22%2C%22979636%22%2C%22984699%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B62731.19140625%2C46027.53125%2C370.08441162109375%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B0.0175847839564085%2C-0.7084158658981323%2C0.016393372789025307%2C0.7053857445716858%5D%2C%22perspectiveZoom%22:1127.027942857649%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D)


### Summary of difficult cases

### 16 mitos (index 5) should be axon compartment (mistake by original Turner et al algorithm)
- these mitos should be axon: 657973,  880606,  657839, 1227495,  880563, 1110781,  772690,
        551428,  659488,  659171,  998238, 1110444, 1226503,  659275,
        771854,  771933
###  9+13 = 22 mitos should be changed to unknown compartment 
- 9 mitos (index 6) and 13 (index 17) could be axon or dendrite so should be moved to "unknown" compartment
- these mitos should be classified as unknown: 
- from index 6: cellid 648518346349532067: 4240479, 4240423, 4240494, 4240472, 4240545, 4240544, 4240389, 4240391, 4118518
- from index 17: cellid 648518346349537513: 1852507, 1852441, 1725469, 1852659, 1852316, 1852546, 1724749, 1725445, 1846859, 1852488, 1724948, 1846747, 1725308
### 161 were impossible to determine so should remain as unknown dendrite
- 2 + 93 + 11 + 55

### Cells that require additional analysis; see next sections
- index 3: cellid 648518346349539845; number of mitos: 171; About half are correctly classified as basal and the remaining belong to apical
- index 12: cellid 648518346349531254; number of mitos: 79; one process is basal and one apical and it is difficult to determine which could be which
- index 21: cellid 648518346349538235; number of mitos: 86; one process is basal and the other is apical


### Additional processing of index 3: cellid 648518346349539845; number of mitos: 171

In [438]:
# analysis of index 3: cellid 648518346349539845; number of mitos: 171
# there are two processes with a good separation at x_position = 103,000
# a few of the mitochondria that should be classified as soma and not dendritic

index3_left = query[query['ctr_pos_x_vx'] < 103000] # This looks like the apical dendrite
index3_right = query[query['ctr_pos_x_vx'] >= 103000] # This looks like a basal dendrite with a few soma mito as well

In [439]:
len(index3_left)

46

In [440]:
len(index3_right)

125

In [441]:
# Reveal index3_left in neuroglancer

cellid_iloc = 3

list_of_cellids = unknown_dendritic_df_basal.cellid.unique()
mito = unknown_dendritic_df_basal
lookup = list_of_cellids[cellid_iloc]
queryx = index3_left #mito[mito["cellid"] == lookup]
query_listx = queryx["mito_id"].to_list()
spacer = "%22%2C%22"
query_list_joinx = spacer.join(map(str,query_listx))
baseURL = 'https://neuromancer-seung-import.appspot.com/'
nglink_a = '#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22'
nglink_b = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%22'
nglink_c = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B83222.921875%2C52981.34765625%2C824.9962768554688%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B-0.00825042650103569%2C0.06130112707614899%2C-0.0012821174459531903%2C0.9980843663215637%5D%2C%22perspectiveZoom%22:3618.7659948513424%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D'
neuroglancerURL = baseURL + nglink_a + str(lookup) + nglink_b + query_list_joinx + nglink_c
# uncomment to show link: print(neuroglancerURL)

In [442]:
np.array(query_listx)

array([3156851, 3045291, 2396394, 3151287, 3049781, 3160223, 3053944,
       3164597, 3156380, 3040095, 2928804, 2618034, 2508530, 3268978,
       2928182, 2937928, 3040180, 3045034, 3268755, 3160838, 3150973,
       3268924, 2953478, 2395545, 2733190, 3061350, 3155898, 3160012,
       3146767, 2846190, 2512383, 3269003, 3151315, 2508540, 3164603,
       2509043, 3062189, 3264327, 3062418, 3160144, 3039415, 3159857,
       2938493, 3264711, 3160540, 2400006])

#### These mito should be classified as apical (the rfc classifier was incorrect) n = 46
3156851, 3045291, 2396394, 3151287, 3049781, 3160223, 3053944,
3164597, 3156380, 3040095, 2928804, 2618034, 2508530, 3268978,
2928182, 2937928, 3040180, 3045034, 3268755, 3160838, 3150973,
3268924, 2953478, 2395545, 2733190, 3061350, 3155898, 3160012,
3146767, 2846190, 2512383, 3269003, 3151315, 2508540, 3164603,
2509043, 3062189, 3264327, 3062418, 3160144, 3039415, 3159857,
2938493, 3264711, 3160540, 2400006

In [446]:
index3_right

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment,centroid_x,centroid_y,centroid_z,ctr_distance_to_soma,bbox_dir_x,bbox_dir_y,bbox_dir_z,bbox_distance,above_soma,within_threshold_1500,count,mean,std,min,25%,50%,75%,max,kurtosis,skew,rfc_predicted_compartment,cellid_str
89,3941827,58008,112768,46896,1432,112498,46816,1430,113024,46946,1438,648518346349539845,451072,187584,57280,0.979014,6.719216,Unknown dendritic,103776,47792,1558,9037.408699,-526,-130,-8,541.885597,1,0,642.0,11570.545512,6548.159019,0.0,7678.267110,9756.752456,14578.065405,33382.562514,3.411755,1.609587,Basal,648518346349539845
90,4059787,101560,116470,45472,1512,116054,45360,1496,116880,45556,1521,648518346349539845,465880,181888,60480,1.688904,11.253759,Unknown dendritic,103776,47792,1558,12904.346245,-826,-196,-25,849.303833,1,0,642.0,14761.295282,6934.517695,0.0,11242.210001,13473.665405,17953.187097,36914.161632,2.218695,1.212875,Basal,648518346349539845
91,3496644,52172,105706,48162,1551,105604,48120,1548,105834,48212,1555,648518346349539845,422824,192648,62040,0.635454,2.271467,Unknown dendritic,103776,47792,1558,1965.158772,-230,-92,-7,247.816464,0,0,642.0,7501.190497,6366.899743,0.0,2495.532744,5441.356462,11124.413910,27724.700918,4.278167,1.930296,Basal,648518346349539845
92,3496814,5976,105806,48744,1549,105782,48724,1547,105838,48766,1552,648518346349539845,423224,194976,61960,0.109632,0.889036,Unknown dendritic,103776,47792,1558,2242.160788,-56,-42,-5,70.178344,0,0,642.0,7533.230618,6222.727719,0.0,2678.658577,5587.393721,11190.185308,27291.483379,4.168325,1.901264,Basal,648518346349539845
93,4040693,268280,114690,41014,1026,114604,40722,966,114796,41474,1092,648518346349539845,458760,164056,41040,3.724692,17.299063,Unknown dendritic,103776,47792,1558,12858.448740,-192,-752,-126,786.284936,1,0,642.0,15233.454089,7220.703687,0.0,11539.048909,13575.126705,17991.711379,36490.191408,2.035260,1.137291,Basal,648518346349539845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,4092278,280284,114554,62612,839,114470,61872,824,114660,63296,849,648518346349539845,458216,250448,33560,3.763039,16.343598,Unknown dendritic,103776,47792,1558,18338.883417,-190,-1424,-25,1436.837152,0,0,642.0,18205.335779,5723.499894,0.0,16328.190172,17944.399059,20336.594034,41641.105377,1.294816,0.897342,Basal,648518346349539845
256,3932008,648328,113792,42062,1520,112190,41446,1507,114994,42632,1538,648518346349539845,455168,168248,60800,7.821484,27.428755,Unknown dendritic,103776,47792,1558,11539.263408,-2804,-1186,-31,3044.663036,1,0,642.0,14045.128241,7135.237479,0.0,10221.147237,12277.337954,16942.633897,35292.385255,2.414222,1.272209,Basal,648518346349539845
257,3820456,340040,111628,42756,1544,111308,42688,1523,111968,42816,1555,648518346349539845,446512,171024,61760,2.697519,4.090361,Unknown dendritic,103776,47792,1558,9328.204329,-660,-128,-32,673.058690,1,0,642.0,12317.413612,6954.725800,0.0,8178.818917,10179.631312,15274.698356,33021.458190,2.937324,1.459688,Basal,648518346349539845
258,3833033,390148,110210,48236,1293,110128,48162,1210,110306,48336,1401,648518346349539845,440840,192944,51720,4.766243,17.139492,Unknown dendritic,103776,47792,1558,6454.743759,-178,-174,-191,313.753088,0,0,642.0,9700.647440,6171.250549,0.0,5715.933399,7600.746286,12458.688965,31284.166107,4.205628,1.855280,Basal,648518346349539845


In [447]:
# Reveal index3_right in neuroglancer

cellid_iloc = 3

list_of_cellids = unknown_dendritic_df_basal.cellid.unique()
mito = unknown_dendritic_df_basal
lookup = list_of_cellids[cellid_iloc]
queryy = index3_right #mito[mito["cellid"] == lookup]
query_listy = queryy["mito_id"].to_list()
spacer = "%22%2C%22"
query_list_joiny = spacer.join(map(str,query_listy))
baseURL = 'https://neuromancer-seung-import.appspot.com/'
nglink_a = '#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22'
nglink_b = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%22'
nglink_c = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B83222.921875%2C52981.34765625%2C824.9962768554688%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B-0.00825042650103569%2C0.06130112707614899%2C-0.0012821174459531903%2C0.9980843663215637%5D%2C%22perspectiveZoom%22:3618.7659948513424%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D'
neuroglancerURL = baseURL + nglink_a + str(lookup) + nglink_b + query_list_joiny + nglink_c
# uncomment to show link: 
print(neuroglancerURL)

https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349539845%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmenta

In [448]:
index3_right_soma = index3_right[index3_right['ctr_pos_x_vx'] < 107000] # isolates the mitos that should be classified as soma
index3_right_basal = index3_right[index3_right['ctr_pos_x_vx'] >= 107000] # remaining mitos that are correctly classified as basal

In [452]:
len(index3_right_soma)

13

In [450]:
np.array(index3_right_soma.mito_id.to_list())

array([3496644, 3496814, 3496828, 3496744, 3607698, 3496331, 3496648,
       3496728, 3496572, 3496556, 3496570, 3496471, 3496328])

#### These mito ids should have been classified as soma mitos (error in original Turner et al Cell 2022 algorithm) n = 13
3496644, 3496814, 3496828, 3496744, 3607698, 3496331, 3496648,
3496728, 3496572, 3496556, 3496570, 3496471, 3496328

In [453]:
len(index3_right_basal)

112

In [451]:
np.array(index3_right_basal.mito_id.to_list())

array([3941827, 4059787, 4040693, 4070614, 4058480, 3936768, 4075025,
       3949395, 3955492, 4172529, 4168259, 3833333, 4058467, 4076917,
       3713885, 3960607, 3709135, 3847636, 3953813, 3717384, 4058387,
       4061872, 4171321, 3720510, 4040773, 3822987, 3961936, 3721532,
       3933854, 3833910, 3933041, 3833749, 3843033, 3833739, 3831054,
       3704552, 3831214, 4048890, 3834058, 3961840, 3967554, 4171865,
       3826826, 3831057, 4092057, 3833836, 3717535, 3953546, 4059263,
       4199846, 3966607, 4086642, 3966618, 4040792, 4087211, 4190863,
       4058849, 3948746, 3822218, 3722165, 3704079, 4052194, 3722223,
       3950619, 3838063, 3721119, 3843067, 3824410, 3822766, 4044162,
       3822477, 3699039, 4087393, 4048786, 4168007, 4205418, 3967536,
       3817730, 3831125, 3704323, 4050856, 3956689, 4048245, 4076106,
       3721249, 3831115, 4157984, 3831150, 4080725, 4058046, 3957233,
       3826388, 3953165, 3708831, 4294603, 3947868, 3953792, 4189353,
       4044134, 3825

#### These mito ids were correctly classified by rfc as basal n = 112
3941827, 4059787, 4040693, 4070614, 4058480, 3936768, 4075025,
       3949395, 3955492, 4172529, 4168259, 3833333, 4058467, 4076917,
       3713885, 3960607, 3709135, 3847636, 3953813, 3717384, 4058387,
       4061872, 4171321, 3720510, 4040773, 3822987, 3961936, 3721532,
       3933854, 3833910, 3933041, 3833749, 3843033, 3833739, 3831054,
       3704552, 3831214, 4048890, 3834058, 3961840, 3967554, 4171865,
       3826826, 3831057, 4092057, 3833836, 3717535, 3953546, 4059263,
       4199846, 3966607, 4086642, 3966618, 4040792, 4087211, 4190863,
       4058849, 3948746, 3822218, 3722165, 3704079, 4052194, 3722223,
       3950619, 3838063, 3721119, 3843067, 3824410, 3822766, 4044162,
       3822477, 3699039, 4087393, 4048786, 4168007, 4205418, 3967536,
       3817730, 3831125, 3704323, 4050856, 3956689, 4048245, 4076106,
       3721249, 3831115, 4157984, 3831150, 4080725, 4058046, 3957233,
       3826388, 3953165, 3708831, 4294603, 3947868, 3953792, 4189353,
       4044134, 3825073, 3831031, 3831060, 3842953, 3720705, 3961237,
       4048125, 4087056, 4092278, 3932008, 3820456, 3833033, 3830889

### additional processing of index 12: cellid 648518346349531254;  number of mitos: 79

In [462]:
# analysis of index 12: cellid 648518346349531254;  number of mitos: 79
# there are two processes with a good separation at x_position = 77,500

index12_left = query[query['ctr_pos_x_vx'] < 77500] # This looks like the apical dendrite
index12_right = query[query['ctr_pos_x_vx'] >= 77500] # This looks like a basal dendrite

In [463]:
len(index12_left)

24

In [464]:
len(index12_right)

55

In [466]:
# These should be assigned apical (the rfc classifier was incorrect) n = 24
np.array(index12_left.mito_id.to_list())

array([1768668, 1890582, 1773075, 1189911, 1885913, 1778442, 1650211,
       2010407, 1778402, 1301533, 1891420, 1190204, 1772953, 1891080,
       1300841, 1299977, 1767137, 1648586, 1891196, 1886041, 1648967,
       1777820, 1533066, 1417199])

In [467]:
# These should be assigned basal (the rfc classifier was correct) n = 55
np.array(index12_right.mito_id.to_list())

array([3591662, 2245089, 3481010, 2243067, 3591624, 3481657, 2131044,
       2484183, 2243516, 2359891, 2359574, 2361621, 2131052, 2243340,
       2372677, 2372434, 2706882, 2825268, 2591923, 2243111, 2131054,
       2130989, 2246789, 3263377, 2238180, 2232805, 2243337, 2598246,
       2130947, 2246159, 2245034, 2238455, 2010839, 2245938, 2608611,
       2238315, 2728528, 2010520, 2127971, 2251186, 2478162, 2130782,
       2824951, 3370551, 2246163, 2251264, 3369210, 2607782, 2245148,
       3153590, 2128360, 2245823, 2705507, 2475672, 2244740])

### Additional processing of index 21: cellid 648518346349538235; number of mitos: 86
Same separation point as index 12!  
See in [Neuroglancer](https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22colorSeed%22:1278570284%2C%22segments%22:%5B%22648518346349531254%22%2C%22648518346349538235%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22colorSeed%22:4131854015%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22colorSeed%22:3080909682%2C%22segments%22:%5B%221085740%22%2C%221200700%22%2C%221294492%22%2C%221301081%22%2C%221313348%22%2C%221423267%22%2C%221426237%22%2C%221427492%22%2C%221427974%22%2C%221429058%22%2C%221429169%22%2C%221430191%22%2C%221430283%22%2C%221430335%22%2C%221430744%22%2C%221548754%22%2C%221548987%22%2C%221549392%22%2C%221550799%22%2C%221664841%22%2C%221665919%22%2C%221670409%22%2C%221671421%22%2C%221671612%22%2C%221671719%22%2C%221783920%22%2C%221784076%22%2C%221788475%22%2C%221789310%22%2C%221789919%22%2C%222127620%22%2C%222131721%22%2C%222131795%22%2C%222132502%22%2C%222132530%22%2C%222135945%22%2C%222137607%22%2C%222137636%22%2C%222239853%22%2C%222249587%22%2C%222249873%22%2C%222249960%22%2C%222250108%22%2C%222250317%22%2C%222250463%22%2C%222254075%22%2C%222254127%22%2C%222254137%22%2C%222254503%22%2C%222254649%22%2C%222254817%22%2C%222355422%22%2C%222361375%22%2C%222361537%22%2C%222364459%22%2C%222366385%22%2C%222367180%22%2C%222368012%22%2C%222472501%22%2C%222480439%22%2C%222480572%22%2C%222481723%22%2C%222590705%22%2C%222591330%22%2C%222591502%22%2C%222594972%22%2C%222705414%22%2C%222706638%22%2C%222711495%22%2C%222828667%22%2C%222940168%22%2C%223157827%22%2C%223158855%22%2C%223376180%22%2C%223489544%22%2C%223600599%22%2C%223714251%22%2C%22545202%22%2C%22546333%22%2C%22649815%22%2C%22745428%22%2C%22745743%22%2C%22746186%22%2C%22854181%22%2C%22867111%22%2C%22974567%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22colorSeed%22:3424028769%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B77527.8671875%2C48919.4453125%2C750.2664184570312%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B-0.00825042650103569%2C0.06130112707614899%2C-0.0012821174459531903%2C0.9980843663215637%5D%2C%22perspectiveZoom%22:1809.3829974256712%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22cell_segmentation_v185%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D) how these two neurons are close to one another

In [477]:
# analysis of index 21: cellid 648518346349538235; number of mitos: 86
# there are two processes with a good separation at x_position = 77,500
# Coincidentally, the same separation point as index 12!
# This one is tough to call; I picked the "right" side as the apical dendrite due to more vertical orientation, slightly larger diameter, and slightly shorter mitos than the putative basal dendrite on the "left"

index21_left = query[query['ctr_pos_x_vx'] < 77500] # I'm assigning this as a basal dendrite
index21_right = query[query['ctr_pos_x_vx'] >= 77500] # I'm assigning this as the apical dendrite

In [478]:
len(index21_left)

39

In [479]:
len(index21_right)

47

In [480]:
# These should be assigned basal (the rfc classifier was correct) n = 39
np.array(index12_left.mito_id.to_list())

array([1768668, 1890582, 1773075, 1189911, 1885913, 1778442, 1650211,
       2010407, 1778402, 1301533, 1891420, 1190204, 1772953, 1891080,
       1300841, 1299977, 1767137, 1648586, 1891196, 1886041, 1648967,
       1777820, 1533066, 1417199])

In [481]:
# These should be assigned apical (the rfc classifier was incorrect) n = 47
np.array(index12_left.mito_id.to_list())

array([1768668, 1890582, 1773075, 1189911, 1885913, 1778442, 1650211,
       2010407, 1778402, 1301533, 1891420, 1190204, 1772953, 1891080,
       1300841, 1299977, 1767137, 1648586, 1891196, 1886041, 1648967,
       1777820, 1533066, 1417199])

# Summary of rfc for basal assignments
### Classified as basal and human inspection confirms basal assignment: 521+112+55+39 = 727
- index 3: 112 mitos were correctly classified as basal
- index 12: 55 mitos were correctly classified as basal
- index 21: 39 mitos were correctly classified as basal 

### Classified as basal and should have been classified as apical: 33 +46+24+47 = 150
- index 3: 46 mitos should have been classified as apical
- index 12: 24 mitos should have been classified as apical
- index 21: 47 mitos should have been classified as apical  

### Edge of volume neurons that can't be determined and should remain as unknown dendrite: 161

### Human inspection showed that the original classification by Turner et al was incorrect: 16+22+13 = 51
- 16 mitos from index 5 are axon (and should not have been in the unknown dendrite category): 
- 22 mitos from index 6 and 17 should be reassigned to unknown compartment since these could be either axon or dendrite
- 13 mitos from index 3 should have been classified as soma in the original Turner dataset

### Total: 727+150+161+51 = 1089
All basal assignments accounted for!

## Accuracy of rfc for basal assignments
- 727/1089 (0.6676) were correctly assigned as basal
- 150/1089 (0.1377) were assigned basal and should have been assigned apical
- 161/1089 (0.1478) were assigned as basal but the assignment cannot be verified due to edge problem
- 51/1089 (0.0468) were assigned as basal but the original dataset assignment was incorrect to begin with

## Aggregate
- 727/1089 (0.668) correct
- 150+161+51/1089 = (1-0.668) = 0.332 incorrect

# Summary of apical and basal assignment accuracy by rfc
Round 1

## Apical
- 11/48 (0.229) classified as apical were true apical upon manual validation  
- 37/48 (0.771) classified as apical were actually basal dendrites as determined by manual inspection

## Basal
- 727/1089 (0.668) classified as basal were true basal upon manual validation  
- 362/1089 = (1-0.668) = 0.332 classified as basal were actually other assignments (apical, axon, soma, unknown, or should remain unknown dendrite)

# Overall accuracy of Random Forest Classifier (rfc)
## Correct assignments: 11+727 = 738/1137 (0.649)
## Incorrect assignments: 37+362 = 399/1137 (0.351)
totals: 48+1089 = 1137

# Overall accuracy of Random Forest Classifier (rfc)
- Correct assignments:   738/1137 (0.649)
- Incorrect assignments: 399/1137 (0.351)


### Reveal in Neuroglancer one mito at a time

In [36]:
baseURL = 'https://neuromancer-seung-import.appspot.com/'
nglink_a = '#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.1%2C%22segments%22:%5B%22'
nglink_b = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%2C%22visible%22:false%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%22'
nglink_c = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%2C%22visible%22:false%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B'
nglink_d = '%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B-0.00825042650103569%2C0.06130112707614899%2C-0.0012821174459531903%2C0.9980843663215637%5D%2C%22perspectiveZoom%22:904.6914987128356%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D'

In [29]:
iloc_value = 0

In [30]:
mito_id_lookup = unknown_dendritic_df.mito_id[iloc_value]
mito_id_lookup

1547760

In [31]:
# match var name with my neuroglancer code
mito = unknown_dendritic_df

In [32]:
mito_id_query = mito["mito_id"] == mito_id_lookup
mito_id_query = mito[mito_id_query]
mito_id_query

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment,centroid_x,centroid_y,centroid_z,ctr_distance_to_soma,bbox_dir_x,bbox_dir_y,bbox_dir_z,bbox_distance,above_soma,within_threshold_1500,count,mean,std,min,25%,50%,75%,max,kurtosis,skew,rfc_predicted_compartment,cellid_str
0,1547760,211468,70706,45080,1364,70248,44874,1328,71030,45284,1393,648518346349523993,282824,180320,54560,2.71677,10.804325,Unknown dendritic,70864,42368,24,3029.1101,-782,-410,-65,885.352472,0,0,196.0,6307.454632,5294.853299,0.0,2791.498779,4378.841631,8158.576625,26607.713618,5.541097,2.220513,Basal,648518346349523993


In [33]:
cellid_of_interest = mito_id_query["cellid"].to_string(index=False)
mitoid_of_interest = mito_id_query["mito_id"].to_string(index=False)
mito_ctr_pos_x_vx = mito_id_query["ctr_pos_x_vx"].to_string(index=False)
mito_ctr_pos_y_vx = mito_id_query["ctr_pos_y_vx"].to_string(index=False)
mito_ctr_pos_z_vx = mito_id_query["ctr_pos_z_vx"].to_string(index=False)

In [34]:
cellid_of_interest

'648518346349523993'

In [37]:
neuroglancerURL = baseURL + nglink_a + str(cellid_of_interest) + nglink_b + str(mitoid_of_interest) + nglink_c + str(mito_ctr_pos_x_vx) + '%2C' + str(mito_ctr_pos_y_vx) + '%2C' + str(mito_ctr_pos_z_vx) + nglink_d

In [38]:
print(neuroglancerURL)

https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.1%2C%22segments%22:%5B%22648518346349523993%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%2C%22visible%22:false%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%